In [14]:
import pandas as pd
import numpy as np

import requests

frost_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'
frost_secert = '4fbc591a-214d-4066-a7a2-d268cf98f3eb'


In [15]:
def get_json(endpoint,params = {},clint_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    r = requests.get(endpoint, params, auth=(clint_id,''))
    json = r.json()
    if r.status_code == 200:
        data = json['data']
        print('Data retrieved from frost.met.no')
    else:
        print('Error! Returned status code %s' % r.status_code)
        print('Message: %s' % json['error']['message'])
        print('Reason: %s' % json['error']['reason'])
    return(data)


def get_df(endpoint,params = {},client_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    #from example code, needs to be tailored for each call (not good)
    data = get_json(endpoint,params,client_id)
    df = pd.DataFrame()
    for i in range(len(data)):
        row = pd.DataFrame(data[i]['observations'])
        row['referenceTime'] = data[i]['referenceTime']
        row['sourceId'] = data[i]['sourceId']
        df = df.append(row)

    return(df.reset_index())
    

def get_df(json):
    #not really tested enough
    df = pd.io.json.json_normalize(json)
    
    return(df)
    

In [16]:
station_json = get_json('https://frost.met.no/sources/v0.jsonld?country=Norge')
station_df = get_df(station_json)


Data retrieved from frost.met.no


In [50]:
# Clean station df
valid_from_date = pd.to_datetime('2018-01-01')


station_df = station_df[~station_df['geometry.coordinates'].isnull()]
station_df.validFrom = pd.to_datetime(station_df.validFrom,errors = 'coerce')
station_df = station_df[((station_df.validFrom < valid_from_date) | (station_df.validFrom.isnull()))]

In [5]:
station_df.to_csv('../csv/stations.csv')

In [62]:
error_idx = [i for i,row in station_df.iterrows() if row['id'] in error_ids]

In [70]:
station_df.head(10)

,@type,country,countryCode,county,countyId,externalIds,geometry.@type,geometry.coordinates,geometry.nearest,icaoCodes,...,masl,municipality,municipalityId,name,shipCodes,shortName,stationHolders,validFrom,wigosId,wmoId
0,SensorSystem,Norge,NO,HEDMARK,4.0,NaN,Point,"[12.5039, 61.1349]",False,NaN,...,333.0,TRYSIL,428.0,PLASSEN,NaN,Plassen,[MET.NO],1968-07-01,0-578-0-100,NaN
1,SensorSystem,Norge,NO,HEDMARK,4.0,NaN,Point,"[11, 62.42]",False,NaN,...,565.0,TOLGA,436.0,TOLGA,NaN,NaN,NaN,NaT,NaN,NaN
2,SensorSystem,Norge,NO,HEDMARK,4.0,[204],Point,"[11.1723, 62.5003]",False,NaN,...,600.0,OS,441.0,FV30 MOSENGET,NaN,Mosenget,[STATENS VEGVESEN],2011-03-14,0-578-0-10070,NaN
3,SensorSystem,Norge,NO,HEDMARK,4.0,NaN,Point,"[11.23, 62.5]",False,NaN,...,600.0,OS,441.0,OS,NaN,NaN,NaN,NaT,NaN,NaN
4,SensorSystem,Norge,NO,TRØNDELAG,50.0,NaN,Point,"[11.7752, 62.4698]",False,NaN,...,650.0,RØROS,5025.0,HÅSJØEN - SOLGLØTT,NaN,Håsjøen,[MET.NO],1997-09-01,0-578-0-10300,NaN
5,SensorSystem,Norge,NO,TRØNDELAG,50.0,"[01288, 10.249.0.69, ENRO]",Point,"[11.3518, 62.5773]",False,[ENRO],...,625.0,RØROS,5025.0,RØROS LUFTHAVN,NaN,Røros,[AVINOR],1957-08-01,0-20000-0-01288,1288.0
7,SensorSystem,Norge,NO,ØSTFOLD,1.0,NaN,Point,"[11, 59.02]",False,NaN,...,30.0,HVALER,111.0,HVALER - SKJÆRHALLEN,NaN,Skjærhallen,NaN,NaT,NaN,NaN
8,SensorSystem,Norge,NO,TRØNDELAG,50.0,NaN,Point,"[11.4534, 62.6737]",False,NaN,...,685.0,RØROS,5025.0,AURSUND,NaN,Aursund,[GLOMMENS OG LAAGENS BRUKSEIERFORENING],1934-05-01,0-578-0-10600,NaN
9,SensorSystem,Norge,NO,TRØNDELAG,50.0,NaN,Point,"[11.83, 62.7]",False,NaN,...,750.0,RØROS,5025.0,AURSUNDEN - SØLENDET,NaN,NaN,NaN,NaT,NaN,NaN
11,SensorSystem,Norge,NO,ØSTFOLD,1.0,NaN,Point,"[11.0444, 59.0367]",False,NaN,...,17.0,HVALER,111.0,HVALER,NaN,Hvaler,[MET.NO],1908-11-01,0-578-0-1080,NaN


In [64]:
station_df.loc[error_idx]

,@type,country,countryCode,county,countyId,externalIds,geometry.@type,geometry.coordinates,geometry.nearest,icaoCodes,...,masl,municipality,municipalityId,name,shipCodes,shortName,stationHolders,validFrom,wigosId,wmoId
36,SensorSystem,Norge,NO,HEDMARK,4.0,[501151140],Point,"[11.1333, 60.767]",False,NaN,...,225.0,STANGE,417.0,ARSTADTAJET,NaN,Arstadtajet,[STANGE KOMMUNE],2017-08-17,NaN,NaN
143,SensorSystem,Norge,NO,OSLO,3.0,[506131083],Point,"[10.8737, 59.9652]",False,NaN,...,201.0,OSLO,301.0,OSLO - AMMERUD,NaN,Ammerud,"[OSLO KOMMUNE, VAV ROSIM]",2013-12-31,0-578-0-18245,NaN
160,SensorSystem,Norge,NO,OSLO,3.0,NaN,Point,"[10.73, 59.91]",False,NaN,...,15.0,OSLO,301.0,OSLO - STUDENTERLUNDEN,NaN,NaN,NaN,NaT,NaN,NaN
180,SensorSystem,Norge,NO,AKERSHUS,2.0,[501151108],Point,"[10.4966, 59.86635]",False,NaN,...,30.0,ASKER,220.0,ASKER - NESBRU,NaN,Nesbru,[ASKER KOMMUNE],2015-03-19,0-578-0-19670,NaN
208,SensorSystem,Norge,NO,OPPLAND,5.0,NaN,Point,"[10.52, 60.42]",False,NaN,...,190.0,GRAN,534.0,BRANDBU,NaN,NaN,NaN,NaT,NaN,NaN
224,SensorSystem,Norge,NO,OPPLAND,5.0,NaN,Point,"[9.55, 60.82]",False,NaN,...,225.0,SØR-AURDAL,540.0,BAGN,NaN,NaN,NaN,NaT,NaN,NaN
385,SensorSystem,Norge,NO,VESTFOLD,7.0,NaN,Point,"[10.03, 59.05]",False,NaN,...,30.0,LARVIK,712.0,LARVIK,NaN,NaN,NaN,NaT,NaN,NaN
389,SensorSystem,Norge,NO,ØSTFOLD,1.0,NaN,Point,"[11.032, 59.1895]",False,NaN,...,10.0,FREDRIKSTAD,106.0,BORGE - BERG,NaN,NaN,[FREDRIKSTAD KOMMUNE],2002-01-01,0-578-0-3015,NaN
390,SensorSystem,Norge,NO,ØSTFOLD,1.0,NaN,Point,"[10.945, 59.2105]",False,NaN,...,10.0,FREDRIKSTAD,106.0,FREDRIKSTAD BRANNSTASJON,NaN,NaN,[FREDRIKSTAD KOMMUNE],2002-04-01,0-578-0-3019,NaN
423,SensorSystem,Norge,NO,ØSTFOLD,1.0,NaN,Point,"[10.92, 59.23]",False,NaN,...,10.0,FREDRIKSTAD,106.0,FREDRIKSTAD,NaN,NaN,NaN,NaT,NaN,NaN


In [45]:
station_df.validFrom[1] == pd._libs.tslib.NaTType

False

In [57]:
error_ids = ['SN18670',
             'SN82220',
             'SN93090',
             'SN3050',
             'SN18245',
             'SN87105',
             'SN84701',
             'SN3019',
             'SN4300',
             'SN47250',
             'SN19670',
             'SN70740',
             'SN39201',
             'SN62510',
             'SN65145',
             'SN53690',
             'SN44660',
             'SN22810',
             'SN84060',
             'SN85545',
             'SN62960',
             'SN30010',
             'SN3015',
             'SN12210',
             'SN76260',
             'SN66720',
             'SN19670',
             'SN38320',
             'SN84770',
             'SN19670',
             'SN34190',
             'SN84930',
             'SN27162',
             'SN93090',
             'SN71580',
             'SN20610',
             'SN30275',
             'SN61045',
             'SN30285',
             'SN94245',
             'SN92770']